Neural network using Keras

In [32]:
import pandas as pd
import numpy as np
np.random.seed(20)

In [33]:
train_data = pd.read_csv("featuresets/3/train_data_one_hot.csv",index_col=0)
test_data = pd.read_csv("featuresets/3/test_data_one_hot.csv",index_col=0)
submissions = pd.read_csv("data/sample_submission.csv")

In [71]:
x_train = train_data.drop("loss",axis=1).as_matrix()
y = np.log(train_data["loss"]).as_matrix()
x_test = test_data.as_matrix()

In [72]:
from sklearn.cross_validation import KFold
nfolds = 5
folds = KFold(len(y), n_folds = nfolds, shuffle = True, random_state = 20)

In [83]:
import os
def write_submission_file(predictions,name):
    if not os.path.exists("results"):
        os.makedirs("results")
    submissions.iloc[:,1] = predictions
    submissions.to_csv('results/' + name + '.csv', index=None)

### Define neural network

In [73]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU

def nn_model(x_train_):
    model = Sequential()
    model.add(Dense(400, input_dim = x_train_.shape[1], init = 'he_normal'))
    model.add(PReLU())
    model.add(Dropout(0.4))
    model.add(Dense(200, init = 'he_normal'))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(Dense(1, init = 'he_normal'))
    model.compile(loss = 'mae', optimizer = 'adadelta')
    return(model)


In [79]:
n_epochs = 50
fit = model.fit(x_train,y, nb_epoch = n_epochs,batch_size=250,verbose = 1)
pred = model.predict(x_test,verbose=1)

Epoch 1/50
188318/188318 [==============================] - 10s - loss: 0.5566    
Epoch 2/50
188318/188318 [==============================] - 10s - loss: 0.5571    
Epoch 3/50
188318/188318 [==============================] - 10s - loss: 0.5514    
Epoch 4/50
188318/188318 [==============================] - 10s - loss: 0.5427    
Epoch 5/50
188318/188318 [==============================] - 11s - loss: 0.5464    
Epoch 6/50
188318/188318 [==============================] - 11s - loss: 0.5400    
Epoch 7/50
188318/188318 [==============================] - 10s - loss: 0.5363    
Epoch 8/50
188318/188318 [==============================] - 9s - loss: 0.5353     
Epoch 9/50
188318/188318 [==============================] - 10s - loss: 0.5295    
Epoch 10/50
188318/188318 [==============================] - 10s - loss: 0.5269    
Epoch 11/50
188318/188318 [==============================] - 10s - loss: 0.5289    
Epoch 12/50
188318/188318 [==============================] - 10s - loss: 0.5224    
E

In [87]:
write_submission_file(np.exp(pred),"keras_nn")

In [ ]:
# Bag NN results
for (train_samples,tune_samples) in folds:
    x_train_f = x_train[train_samples]
    y_train_f = y[train_samples]
    x_tune_f = x_train[tune_samples]
    y_tune_f = y[tune_samples]
    pred = np.zeros(x_tune_f.shape[0])
    
    model = nn_model(x_train_f)
    
    fit = model.fit(x_train_f,y_train_f, nb_epoch = n_epochs,batch_size=10,verbose = 1)
    
    pred = model.predict(x_tune_f,verbose=1)
        
    pred_oob[tune_samples] = pred
    score = mean_absolute_error(y_tune_f, pred)
    i += 1
    print('Fold ', i, '- MAE:', score)
